In [2]:
from resnet import *
from cifar_very_tiny import *
from cifar_dataset import *    
import torch as t 
import numpy as np
import tqdm
import matplotlib.pylab as plt
import matplotlib.cm as cm
%matplotlib inline
plt.rcParams['figure.figsize']=(12,9)
plt.rcParams['font.size']= 20

In [3]:
train_loader, test_loader, train_loader_no_augumentation = cifar10_loader(batch_size=128)

100.0%

Extracting ../data/cifar-10-python.tar.gz to ../data
Files already downloaded and verified
Files already downloaded and verified


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
epoch_num = 50
basic_results = [] # результаты без дистилляции. Каждый элемент списка - accuracy
resnet_results = [] # результаты с дистилляцией ResNet. Каждый элемент списка - кортеж вида (accuracy, beta, temp)
cnn_results = [] # результаты с дистилляцией CNN. Каждый элемент списка - кортеж вида (accuracy, beta, temp)

In [5]:
def accuracy(student):
        total = 0 
        correct = 0
        with t.no_grad():
            for x,y in test_loader:
                x = x.to(device)
                y = y.to(device)
                out = student(x)
                correct += t.eq(t.argmax(out, 1), y).sum()
                total+=len(x)
        return (correct/total).cpu().detach().numpy()

In [6]:
# 5 запусков --- без дистилляции

In [ ]:
for _ in range(5):
    student = Cifar_Very_Tiny(10).to(device)
    optim = t.optim.Adam(student.parameters())
    #optim = t.optim.SGD(student.parameters(), lr=0.001)
    crit = nn.CrossEntropyLoss()
    for _ in range(epoch_num):
        tq = tqdm.tqdm(train_loader_no_augumentation)
        losses = []
        for x,y in tq:
            x = x.to(device)
            y = y.to(device)
            student.zero_grad()
            out = student(x)
            loss = crit(student(x), y)
            losses.append(loss.cpu().detach().numpy())
            loss.backward()
            optim.step()
            tq.set_description('current loss:{}'.format(np.mean(losses[-10:])))
    basic_results.append(accuracy(student))
    print ('accuracy', basic_results[-1])

current loss:1.78475022315979:  36%|███▋      | 142/391 [00:09<00:08, 27.83it/s]  

In [7]:
kl = nn.KLDivLoss(reduction='batchmean')
sm = nn.Softmax(dim=1)

def distill(out, batch_logits, temp):
    g = sm(out/temp)
    f = F.log_softmax(batch_logits/temp)    
    return kl(f, g)

In [12]:
# 20 запусков --- с ResNet
logits = np.load('./logits_resnet.npy')
for _ in range(20):
    beta = np.random.uniform()
    temp = 10**(np.random.uniform(low=-1, high=1)) # температура от 0.1 до 10
    print ('hyperparameters', beta, temp)
    student = Cifar_Very_Tiny(10).to(device)
    optim = t.optim.Adam(student.parameters())
    #optim = t.optim.SGD(student.parameters(), lr=0.001)
    crit = nn.CrossEntropyLoss()
    for _ in range(epoch_num):
        tq = tqdm.tqdm(train_loader_no_augumentation)
        losses = []
        for batch_id, (x,y) in enumerate(tq):
            x = x.to(device)
            y = y.to(device)
            batch_logits = t.Tensor(logits[128*batch_id:128*(batch_id+1)]).to(device)            
            student.zero_grad()
            out = student(x)
            student_loss = crit(student(x), y)
            #distillation_loss = 0.0 # здесь твой код!
            distillation_loss = distill(out, batch_logits, temp)
            loss = (1-beta) * student_loss + beta*distillation_loss
            losses.append(loss.cpu().detach().numpy())
            loss.backward()
            optim.step()
            tq.set_description('current loss:{}'.format(np.mean(losses[-10:])))
    resnet_results.append((accuracy(student), beta, temp))
    print ('accuracy', resnet_results[-1])

  0%|          | 0/391 [00:00<?, ?it/s]

hyperparameters 0.11958219685629079 0.17182169343405432


  0%|          | 0/391 [00:00<?, ?it/s]

accuracy (array(0.51199996, dtype=float32), 0.11958219685629079, 0.17182169343405432)
hyperparameters 0.7932574995506428 2.6571499396075287


  0%|          | 0/391 [00:00<?, ?it/s]

...

accuracy (array(0.3509, dtype=float32), 0.8575734834141558, 2.5363177272872863)
hyperparameters 0.08789189176403112 0.3030171051381871


  0%|          | 0/391 [00:00<?, ?it/s]

accuracy (array(0.525, dtype=float32), 0.08789189176403112, 0.3030171051381871)
hyperparameters 0.5389962357205916 1.85106538587003


current loss:-0.5404899716377258: 100%|██████████| 391/391 [00:11<00:00, 33.96it/s]


accuracy (array(0.45409998, dtype=float32), 0.5389962357205916, 1.85106538587003)


In [ ]:
# 20 запусков --- с CNN
logits = np.load('./logits_cnn.npy')
for _ in range(20):
    beta = np.random.uniform()
    temp = 10**(np.random.uniform(low=-1, high=1)) # температура от 0.1 до 10  
    print ('hyperparameters', beta, temp)
    student = Cifar_Very_Tiny(10).to(device)
    optim = t.optim.Adam(student.parameters())
    #optim = t.optim.SGD(student.parameters(), lr=0.001)
    crit = nn.CrossEntropyLoss()
    for _ in range(epoch_num):
        tq = tqdm.tqdm(train_loader_no_augumentation)
        losses = []
        for batch_id, (x,y) in enumerate(tq):
            x = x.to(device)
            y = y.to(device)            
            batch_logits = t.Tensor(logits[128*batch_id:128*(batch_id+1)]).to(device)            
            student.zero_grad()
            out = student(x)
            student_loss = crit(student(x), y)
            #distillation_loss = 0.0 # здесь твой код!
            distillation_loss = distill(out, batch_logits, temp)
            loss = (1-beta) * student_loss + beta*distillation_loss
            losses.append(loss.cpu().detach().numpy())
            loss.backward()
            optim.step()
            tq.set_description('current loss:{}'.format(np.mean(losses[-10:])))
    cnn_results.append((accuracy(student), beta, temp))
    print ('accuracy', cnn_results[-1])

In [ ]:
# график, разрез beta-accuracy
plt.scatter([0]*len(basic_results), basic_results, c='r', label='Без дистилляции')
#plt.scatter([r[1] for r in cnn_results], [r[0] for r in cnn_results], c='r', marker='x', label='Дистилляция CNN')
plt.scatter([r[1] for r in resnet_results], [r[0] for r in resnet_results], c='r', marker='d', label='Дистилляция ResNet')
plt.legend(loc='best')
plt.xlabel('Beta')
plt.ylabel('Accuracy')
plt.savefig('scatter_beta_acc.png')

In [ ]:
# график, разрез accuracy-Temp
plt.scatter([0.0]*len(basic_results), basic_results, c='r', label='Без дистилляции')
#plt.scatter([np.log10(r[2]) for r in cnn_results], [r[0] for r in cnn_results], c='r', marker='x', label='Дистилляция CNN')
plt.scatter([np.log10(r[2]) for r in resnet_results], [r[0] for r in resnet_results], c='r', marker='d', label='Дистилляция ResNet')
plt.legend(loc='best')
plt.xlabel('log(T)')
plt.ylabel('Accuracy')
plt.savefig('scatter_temp_acc.png')

In [ ]:
# график, разрез Temp-beta. Цвет --- точность относительно всех результатов

# для резульатов без дистилляции у нас будут повторяющиеся точки. 
# обычно в таком случае в график вносят небольшой шум, чтобы точки можно было визуально различить
eps = 0.02 

all_results = basic_results+[r[0] for r in cnn_results] + [r[0] for r in resnet_results]
max_ = np.max(all_results)
min_ = np.min(all_results)
colors = [cm.seismic((r-min_)/(max_-min_)) for r in basic_results]
plt.scatter(np.random.randn(len(basic_results))*eps, np.random.randn(len(basic_results))*eps, c=colors, label='Без дистилляции')
#colors = [cm.seismic((r[0]-min_)/(max_-min_)) for r in cnn_results]
#plt.scatter([np.log10(r[2]) + np.random.randn()*eps for r in cnn_results], [r[1] + np.random.randn()*eps for r in cnn_results], c=colors, marker='x', label='Дистилляция CNN')
#colors = [cm.seismic((r[0]-min_)/(max_-min_)) for r in resnet_results]
#plt.scatter([np.log10(r[2]) + np.random.randn()*eps for r in resnet_results], [r[1] + np.random.randn()*eps for r in resnet_results], c=colors, marker='d', label='Дистилляция ResNet')
plt.legend(loc='best')
plt.xlabel('log(T)')
plt.ylabel('Beta')
plt.savefig('scatter_temp_beta.png')